# <center> $\underline{\textbf{Relatório Ciência de Redes - LNCC}}$ <p><center> Aluno : Flávio Motta <p><center> Professor: Artur Ziviani <p><center> Semestre 02-2020

# 1 - Introdução
Dota 2 é um jogo multijogador no qual dois times (iluminados e temidos) batalham em partidas cinco contra cinco, o objetivo dos dois times é destruir o trono do adversário.
Ao iniciar o jogo, cada jogador deve escolher um herói dentre 113 disponíveis, dessa forma, ao começar a batalha, tem-se dois times com cinco heróis escolhidos arbitrariamente por cada lado, não é possível repetir heróis.
Sempre existe um time vencedor e um perdedor, não há possibilidade de empate no jogo. <p>
No site do Kaggle é possível encontrar databases de partidas desse jogo.
Com base nesssas premissas, é possível construir um grafo onde os nós são os heróis do jogo e a ligação entre eles é feita quando houver derrota pelo mesmo time.
A base de dados utilizada é relativamente grande, são 100 mil partidas, por esse motivo, é provável que ao montar o grafo ele seja bem denso, uma possibilidade é de colocar peso nas arestas e analisar o grafo a partir de um certo peso.
Com base nessa análise, é possível classificar ao se escolher um herói, quais são seus parceiros de time com pior desempenho no geral.

# 2 - Importando biblioteca

A biblioteca pandas do python é utilizada para manipulação de dados em formatos de ${dataframes}$

In [60]:
import pandas as pd

# 3 - Leitura de arquivos
Logo após podemos fazer a leitura de duas bases de dados, o objetivo desse pré processamento de dados é de conseguir unir as duas bases para que tenha-se uma lista de quais heróis compunham um time quando derrotados. <p>
O arquivo 'vitoria.csv' contém um identificador de partida e qual time venceu a partida <p>
O arquivo 'jogadores.csv' contém o mesmo identificador do outro arquivo, um identificador de qual herói foi utilizado por determinado jogador e qual a posição ele ocupou na partida (com essa coluna conseguimos recuperar em qual time ele jogou)

In [61]:
df = pd.read_csv('vitoria.csv')
df2 = pd.read_csv('jogadores.csv')

Faço dois "head" apenas para verificar como o database se apresenta

In [62]:
df.head()

,match_id,radiant_win
0,50000,0
1,50001,1
2,50002,0
3,50003,1
4,50004,0


In [63]:
df2.head()

,match_id,account_id,hero_id,player_slot
0,50000,117784,96,0
1,50000,158361,84,1
2,50000,158362,46,2
3,50000,137970,85,3
4,50000,1090,39,4


Além dos "head" também faço um "describe" para analisar alguns dados do arquivo

In [64]:
df.describe()

,match_id,radiant_win
count,100000.000000,100000.000000
mean,99999.500000,0.518610
std,28867.657797,0.499656
min,50000.000000,0.000000
25%,74999.750000,0.000000
50%,99999.500000,1.000000
75%,124999.250000,1.000000
max,149999.000000,1.000000


Uma coisa interessante de se notar nessa descrição obtida é que existe uma média de 51,82% de vitória para o time que jogar pelos iluminados

# 4 - Fazendo a mescla das bases
<p> Nesse momento do código é possível fazer o ${join}$ das bases de dados utilizando a coluna "match_id"

In [65]:
partida = pd.merge(df, df2, how='left', left_on='match_id', right_on='match_id')

Novamente utilizo a função "head" para verificar como o resultado da operação anterior (muitas vezes uso essa função porque por ela me mostrar apenas as cinco primeiras linhas da base de dados, ela tem uma execução mais leve)

In [66]:
partida.head()

,match_id,radiant_win,account_id,hero_id,player_slot
0,50000,0,117784,96,0
1,50000,0,158361,84,1
2,50000,0,158362,46,2
3,50000,0,137970,85,3
4,50000,0,1090,39,4


A coluna 'account_id' armazena qual o identificador do jogador, para essa análise de dados não utilizo essa informação

In [67]:
partida = partida.drop(['account_id'],axis=1)

In [68]:
partida.head()

,match_id,radiant_win,hero_id,player_slot
0,50000,0,96,0
1,50000,0,84,1
2,50000,0,46,2
3,50000,0,85,3
4,50000,0,39,4


# 5 - Limpando o dataframe
## 5.1 - Organizando os times
No momento meu ${dataframe}$ apresenta a coluna "radiant_win" que representa se houve vitória dos iluminados naquela partida. <p>
Também existe a coluna "player_slot" que mostra qual posição o jogador ocupou na partida. <p>
Como eu quero apenas analisar os heróis que foram derrotados, dada a posição do jogador e quem venceu, eu posso selecionar apenas as linhas que representam os perdedores. <p>
A coluna "match_id" ainda está preservada porque com ela eu consigo unir quais cinco heróis estavam juntos na derrota.

In [69]:
partida.loc[partida.player_slot == 1, "player_slot"] = 0
partida.loc[partida.player_slot == 2, "player_slot"] = 0
partida.loc[partida.player_slot == 3, "player_slot"] = 0
partida.loc[partida.player_slot == 4, "player_slot"] = 0

In [70]:
partida.loc[partida.player_slot == 128, "player_slot"] = 1
partida.loc[partida.player_slot == 129, "player_slot"] = 1
partida.loc[partida.player_slot == 130, "player_slot"] = 1
partida.loc[partida.player_slot == 131, "player_slot"] = 1
partida.loc[partida.player_slot == 132, "player_slot"] = 1

Os valores possíveis na coluna "player_slot" eram distribuidas dessa forma: <p>
    [0,1,2,3,4] para os iluminados <p>
    [128,129,130,131,132] para os temidos <p>
Para facilitar, coloquei todos do time iluminados com valor 0 e todos do temidos com valor 1

In [71]:
partida.describe()

,match_id,radiant_win,hero_id,player_slot
count,1000000.000000,1000000.000000,1000000.000000,1000000.0
mean,99999.500000,0.518610,50.960134,0.5
std,28867.527892,0.499654,33.229504,0.5
min,50000.000000,0.000000,0.000000,0.0
25%,74999.750000,0.000000,21.000000,0.0
50%,99999.500000,1.000000,48.000000,0.5
75%,124999.250000,1.000000,75.000000,1.0
max,149999.000000,1.000000,112.000000,1.0


Describe para ter certeza que os unicos valores na coluna "player_slot" são 0 e 1

In [72]:
print(partida)

        match_id  radiant_win  hero_id  player_slot
0          50000            0       96            0
1          50000            0       84            0
2          50000            0       46            0
3          50000            0       85            0
4          50000            0       39            0
...          ...          ...      ...          ...
999995    149999            0       74            1
999996    149999            0        8            1
999997    149999            0       50            1
999998    149999            0       55            1
999999    149999            0       62            1

[1000000 rows x 4 columns]


In [73]:
vencedores = partida

No print realizado, pode-se perceber que existem 1 milhão de registros. Como o jogo é dividido em 5 jogadores contra 5 jogadores e sempre um time perde, ao excluir as linhas vencedoras, teremos 500 mil registros.

## 5.2 - Removendo os registros vitoriosos
Para eu ficar apenas com os jogadores derrotados, basta remover os vitoriosos, que são: 
<p> Colunas com derrota dos iluminados ("radiant_win" = 0) e jogador do time dos temidos ("player_slot" = 1), ou
<p> Vitória dos iluminados ("radiant_win" = 1) e jogador do time dos iluminados ("player_slot" = 0)

In [74]:
partida = partida.drop(partida[(partida.radiant_win == 0) & (partida.player_slot == 1)].index)
partida = partida.drop(partida[(partida.radiant_win == 1) & (partida.player_slot == 0)].index)

In [75]:
print(partida)

        match_id  radiant_win  hero_id  player_slot
0          50000            0       96            0
1          50000            0       84            0
2          50000            0       46            0
3          50000            0       85            0
4          50000            0       39            0
...          ...          ...      ...          ...
999990    149999            0       86            0
999991    149999            0       11            0
999992    149999            0       32            0
999993    149999            0      106            0
999994    149999            0       28            0

[500000 rows x 4 columns]


Pode-se ver que o número de registros caiu pela metade.
Agora já não precisamos mais das colunas "radiant_win" e "player_slot"

In [76]:
partida = partida.drop(['radiant_win'],axis=1)
partida = partida.drop(['player_slot'],axis=1)

In [77]:
print(partida)

        match_id  hero_id
0          50000       96
1          50000       84
2          50000       46
3          50000       85
4          50000       39
...          ...      ...
999990    149999       86
999991    149999       11
999992    149999       32
999993    149999      106
999994    149999       28

[500000 rows x 2 columns]


## 5.3 - Mesclando nosso dataframe com a base de nome dos heróis
O arquivo "nomesHerois.csv" contém o id relacionado ao nome do herói do jogo

In [78]:
nomesHerois = pd.read_csv('nomesHerois.csv')
nomesHerois = nomesHerois.drop(['name'],axis=1)
nomesHerois.head()

,hero_id,localized_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


Agora podemos fazer o join, dessa forma teremos o nome do herói usado na partida derrotada

In [79]:
partida = pd.merge(partida, nomesHerois, how='left', left_on='hero_id', right_on='hero_id')
partida.head()

,match_id,hero_id,localized_name
0,50000,96,Centaur Warrunner
1,50000,84,Ogre Magi
2,50000,46,Templar Assassin
3,50000,85,Undying
4,50000,39,Queen of Pain


Usando a função groupby em conjunto com a função apply, é possível agrupar as partidas (por id) e listar os heróis utilizados na mesma.
Nesse momento, separei em duas bases de dados, uma que vai alimentar meu grafo (por id), outra para apenas curiosidade da listagem de heróis)

In [80]:
derrotadosPorID = partida.groupby('match_id')['hero_id'].apply(list).reset_index(name='timeDerrotado')

In [81]:
#derrotadosPorID = derrotadosPorID.drop(['match_id'],axis=1)
derrotadosPorID.head()

,match_id,timeDerrotado
0,50000,"[96, 84, 46, 85, 39]"
1,50001,"[58, 61, 21, 18, 14]"
2,50002,"[74, 7, 42, 99, 88]"
3,50003,"[2, 101, 32, 7, 72]"
4,50004,"[98, 26, 73, 51, 46]"


In [82]:
derrotadosPorNome = partida.groupby('match_id')['localized_name'].apply(list).reset_index(name='timeDerrotado')

In [83]:
#derrotadosPorNome = derrotadosPorNome.drop(['match_id'],axis=1)
derrotadosPorNome.head()

,match_id,timeDerrotado
0,50000,"[Centaur Warrunner, Ogre Magi, Templar Assassi..."
1,50001,"[Enchantress, Broodmother, Windranger, Sven, P..."
2,50002,"[Invoker, Earthshaker, Wraith King, Bristlebac..."
3,50003,"[Axe, Skywrath Mage, Riki, Earthshaker, Gyroco..."
4,50004,"[Timbersaw, Lion, Alchemist, Clockwerk, Templa..."


In [52]:
#derrotadosPorID.to_csv('derrotadorPorID.csv')
#derrotadosPorNome.to_csv('derrotadorPorNome.csv')

# 6 - Próximos passos
Ao fim deste código, temos duas base de dados, a que servirá de entrada para o grafo é a 'derrotadosPorID', ela basicamente contém uma lista de cinco heróis derrotados em conjunto.
A outra base criada 'derrotadosPorNome' foi mais por curiosidade para enxergar os heróis em times. <p>
Como próximo passo utilizarei a base de derrotados em um grafo, dessa forma será possível analisar diversas características dessa base de dados, como, quais heróis tem maior índice de derrotas em conjunto, quais heróis não perdem jogando junto e outras.

In [84]:
vencedores = vencedores.drop(vencedores[(vencedores.radiant_win == 0) & (vencedores.player_slot == 0)].index)
vencedores = vencedores.drop(vencedores[(vencedores.radiant_win == 1) & (vencedores.player_slot == 1)].index)

In [85]:
print(vencedores)

        match_id  radiant_win  hero_id  player_slot
5          50000            0        9            1
6          50000            0       75            1
7          50000            0      106            1
8          50000            0       74            1
9          50000            0       62            1
...          ...          ...      ...          ...
999995    149999            0       74            1
999996    149999            0        8            1
999997    149999            0       50            1
999998    149999            0       55            1
999999    149999            0       62            1

[500000 rows x 4 columns]


In [86]:
vencedores = vencedores.drop(['radiant_win'],axis=1)
vencedores = vencedores.drop(['player_slot'],axis=1)

In [87]:
print(vencedores)

        match_id  hero_id
5          50000        9
6          50000       75
7          50000      106
8          50000       74
9          50000       62
...          ...      ...
999995    149999       74
999996    149999        8
999997    149999       50
999998    149999       55
999999    149999       62

[500000 rows x 2 columns]


In [88]:
vencedoresPorID = vencedores.groupby('match_id')['hero_id'].apply(list).reset_index(name='timeVencedor')

In [89]:
vencedoresPorID.head()

,match_id,timeVencedor
0,50000,"[9, 75, 106, 74, 62]"
1,50001,"[44, 10, 57, 2, 106]"
2,50002,"[69, 8, 25, 26, 79]"
3,50003,"[44, 15, 110, 56, 94]"
4,50004,"[2, 106, 50, 65, 21]"


In [90]:
dfFinal = pd.merge(vencedoresPorID, derrotadosPorID, how='left', left_on='match_id', right_on='match_id')
dfFinal.head()

,match_id,timeVencedor,timeDerrotado
0,50000,"[9, 75, 106, 74, 62]","[96, 84, 46, 85, 39]"
1,50001,"[44, 10, 57, 2, 106]","[58, 61, 21, 18, 14]"
2,50002,"[69, 8, 25, 26, 79]","[74, 7, 42, 99, 88]"
3,50003,"[44, 15, 110, 56, 94]","[2, 101, 32, 7, 72]"
4,50004,"[2, 106, 50, 65, 21]","[98, 26, 73, 51, 46]"


In [91]:
dfFinal = dfFinal.drop(['match_id'],axis=1)

In [92]:
dfFinal.to_csv('vitoriaDerrota.csv')